<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/CodificacionVaribles_Codificacion_enteros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Codificación por números enteros

La codificación por enteros consiste en reemplazar las categorías por dígitos o números enteros del 1 a n ( o de 0 a n-1), dependiendo de la implementación, donde n es el número de las distintas categorías de una variable.

Lo números son asignados arbitrariamente. Este método de codificación permite una rápida evaluación comparativa o 'benchmarking' de los modelos de machine learning.


### Ventajas

- Fácil de implementar
- No expande el espacio de los datos (número de variables)

### Limitaciones

- No añade ninguna información que puede hacer que la  variable tenga mas poder predictivo
- No retiene información sobre las etiquetas de las variables.
- No es adecuado para modelos lineales

La codificación por enteros es más adecuada para métodos no-lineales ya que estos pueden explorar los dígitos arbitrariamente asignados y  tratar de encontrar la mejor relación con el target o variable objetivo.

## En este demo:

Vamos a realizar codificación one hot con:
- pandas
- Scikit-learn
- Feature-Engine

y veremos las ventajas y limitaciones de cada una de estas implementaciones usando los datos House Prices dataset. 



In [1]:
pip install feature_engine

     |████████████████████████████████| 9.5MB 7.8MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
import numpy as np
import pandas as pd

# separar datasets
from sklearn.model_selection import train_test_split

# Codificación por enteros con sklearn
from sklearn.preprocessing import LabelEncoder

# Codificación por enteros con feature-engine
from feature_engine.categorical_encoders import OrdinalCategoricalEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# cargar dataset

data = pd.read_csv(
    '/content/drive/My Drive/datasets/houseprice.csv',
    usecols=['Neighborhood', 'Exterior1st', 'Exterior2nd', 'SalePrice'])

data.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice
0,CollgCr,VinylSd,VinylSd,208500
1,Veenker,MetalSd,MetalSd,181500
2,CollgCr,VinylSd,VinylSd,223500
3,Crawfor,Wd Sdng,Wd Shng,140000
4,NoRidge,VinylSd,VinylSd,250000


In [5]:
# miremos cuantas etiquetas tiene cada variable

for col in data.columns:
    print(col, ': ', len(data[col].unique()), ' etiquetas')

Neighborhood :  25  etiquetas
Exterior1st :  15  etiquetas
Exterior2nd :  16  etiquetas
SalePrice :  663  etiquetas


In [6]:
# exploremos las categorías de cada una de las variables

data['Neighborhood'].unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [7]:
data['Exterior1st'].unique()

array(['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock'], dtype=object)

In [8]:
data['Exterior2nd'].unique()

array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock'], dtype=object)

### Importante sobre codificación

Es importante seleccionar cual dígito asignar a cada una de las categorías usando el set de entrenamiento; y luego usar este mapeo para codificar las variables en el set de prueba


In [9]:
# separemos en sets de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # predictores
    data['SalePrice'],  # target
    test_size=0.3,  # porcentaje observaciones prueba
    random_state=0)  # semilla para asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

## Codificación por enteros con pandas

### Ventajas

- rápido
- devuelve un dataframe tipo pandas

### Limitaciones:

- no preserva la información del set de entrenamiento para propagarlo al set de prueba

Necesitamos capturar y guardar los mapeos uno por uno, manualmente, si estamos planeando usarlos en producción

In [10]:
# primero creemos un diccionario con los mapeos de las categorias en dígitos o números enteros

ordinal_mapping = {
    k: i
    for i, k in enumerate(X_train['Neighborhood'].unique(), 0)
}

ordinal_mapping

{'Blmngtn': 22,
 'Blueste': 24,
 'BrDale': 14,
 'BrkSide': 2,
 'ClearCr': 1,
 'CollgCr': 0,
 'Crawfor': 6,
 'Edwards': 3,
 'Gilbert': 10,
 'IDOTRR': 19,
 'MeadowV': 12,
 'Mitchel': 8,
 'NAmes': 7,
 'NPkVill': 20,
 'NWAmes': 15,
 'NoRidge': 18,
 'NridgHt': 16,
 'OldTown': 13,
 'SWISU': 4,
 'Sawyer': 5,
 'SawyerW': 17,
 'Somerst': 11,
 'StoneBr': 21,
 'Timber': 9,
 'Veenker': 23}

El diccionario indica cual número va a reemplazar cada categoría. Los números fueron asignados arbitrariamente del 0 al n-1 donde n es el número de las diferentes categorias

In [11]:
# reemplaza las etiquetas con los enteros

X_train['Neighborhood'] = X_train['Neighborhood'].map(ordinal_mapping)
X_test['Neighborhood'] = X_test['Neighborhood'].map(ordinal_mapping)

In [12]:
# exploremos el resultados

X_train['Neighborhood'].head(10)

64      0
682     1
960     2
1384    3
1100    4
416     5
1034    6
853     7
472     3
1011    3
Name: Neighborhood, dtype: int64

In [13]:
# podemos convertir los comandos anteriores en dos funciones

def find_category_mappings(df, variable):
    return {k: i for i, k in enumerate(df[variable].unique(), 0)}


def integer_encode(train, test, variable, ordinal_mapping):

    X_train[variable] = X_train[variable].map(ordinal_mapping)
    X_test[variable] = X_test[variable].map(ordinal_mapping)

In [15]:
find_category_mappings(X_test,'Neighborhood')

{0: 11,
 1: 3,
 2: 2,
 3: 14,
 4: 22,
 5: 12,
 6: 0,
 7: 1,
 8: 16,
 9: 19,
 10: 10,
 11: 6,
 12: 13,
 13: 5,
 14: 4,
 15: 9,
 16: 18,
 17: 15,
 18: 8,
 19: 7,
 20: 21,
 21: 17,
 22: 20,
 23: 23}

In [16]:
# y ahora repitamos para el resto de las variables categóricas

for variable in ['Exterior1st', 'Exterior2nd']:
    mappings = find_category_mappings(X_train, variable)
    integer_encode(X_train, X_test, variable, mappings)

In [17]:
# veamos el resultado

X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,0,0,0
682,1,1,1
960,2,1,2
1384,3,2,3
1100,4,1,1


## Codificación por enteros con Scikit-learn

In [18]:
# separemos en sets de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # variables
    data['SalePrice'],  # target
    test_size=0.3,  # porcentaje observaciones prueba
    random_state=0)  # semilla para asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [19]:
# creemos el codificador

le = LabelEncoder()
#Aprende de las etiquetas
le.fit(X_train['Neighborhood'])

LabelEncoder()

In [20]:
# veamos las categorías o 'clases' únicas
le.classes_

array(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
       'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel',
       'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown',
       'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber',
       'Veenker'], dtype=object)

In [21]:
X_train['Neighborhood'] = le.transform(X_train['Neighborhood'])
X_test['Neighborhood'] = le.transform(X_test['Neighborhood'])

X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,5,VinylSd,VinylSd
682,4,Wd Sdng,Wd Sdng
960,3,Wd Sdng,Plywood
1384,7,WdShing,Wd Shng
1100,18,Wd Sdng,Wd Sdng


Desafortunadamente, el LabelEncoder solo funciona con una variable. Sin embargo hay una forma de automatizarlos para todas las variables categóricas ( al mismo tiempo) a la vez. La implementación a continaución fue tomada de [stackoverflow thread](https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn)

In [22]:
#  importar diccionario

from collections import defaultdict

In [23]:
# separemos en segmentos de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # variables
    data['SalePrice'],  # target
    test_size=0.3,  # porcentaje de observaciones prueba
    random_state=0)  # semilla para asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [25]:
#Utilizamos el método
d = defaultdict(LabelEncoder)

In [32]:
# codificación de la  variable
#utilizamos la método apply junto con una función lambda para que aprenda y transforme al mismo tiempo las variables 
train_transformed = X_train.apply(lambda x: d[x.name].fit_transform(x))

# # usando el diccionario para codificar datos futuros
test_transformed = X_test.apply(lambda x: d[x.name].transform(x))

In [33]:
train_transformed.head()

,Neighborhood,Exterior1st,Exterior2nd
64,5,12,13
682,4,13,14
960,3,13,10
1384,7,14,15
1100,18,13,14


In [34]:
test_transformed.head()

,Neighborhood,Exterior1st,Exterior2nd
529,6,13,11
491,12,13,14
459,3,8,8
279,4,9,10
655,2,6,7


In [35]:
# y la transformada inversa para recobrar las etiquetas originales

# # Inversa de la codificación

tmp = train_transformed.apply(lambda x: d[x.name].inverse_transform(x))
tmp.head()

,Neighborhood,Exterior1st,Exterior2nd
64,CollgCr,VinylSd,VinylSd
682,ClearCr,Wd Sdng,Wd Sdng
960,BrkSide,Wd Sdng,Plywood
1384,Edwards,WdShing,Wd Shng
1100,SWISU,Wd Sdng,Wd Sdng


Finalmente, hay otro transformador en Scikit-learn, el OrdinalEncoder, para codificar múltiples variables al mismo tiempo. Sin embargo, este transformador retorna un NumPy array sin los nombres de las columnas, por lo tanto no es nuestra implementación favorita. 

Más detalles aquí: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html 

## Codificación por enteros con Feature-Engine

In [36]:
# separemos en sets de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # variables
    data['SalePrice'],  # target
    test_size=0.3,  # porcentaje observaciones prueba
    random_state=0)  # semilla asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [37]:
ordinal_enc = OrdinalCategoricalEncoder(
    encoding_method='arbitrary', #Lo indicamos así para que las enumere de acuerdo a su orden aparición
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])#Indicamos variables a trabajar

ordinal_enc.fit(X_train)

OrdinalCategoricalEncoder(encoding_method='arbitrary',
                          variables=['Neighborhood', 'Exterior1st',
                                     'Exterior2nd'])

In [38]:
# en el atributo encoder_dict podemos ver los números asignados
# a cada categoría para todas las variables indicadas

ordinal_enc.encoder_dict_

{'Exterior1st': {'AsbShng': 11,
  'AsphShn': 5,
  'BrkComm': 10,
  'BrkFace': 6,
  'CBlock': 13,
  'CemntBd': 8,
  'HdBoard': 3,
  'ImStucc': 12,
  'MetalSd': 4,
  'Plywood': 7,
  'Stone': 14,
  'Stucco': 9,
  'VinylSd': 0,
  'Wd Sdng': 1,
  'WdShing': 2},
 'Exterior2nd': {'AsbShng': 12,
  'AsphShn': 6,
  'Brk Cmn': 13,
  'BrkFace': 8,
  'CBlock': 14,
  'CmentBd': 7,
  'HdBoard': 4,
  'ImStucc': 10,
  'MetalSd': 5,
  'Other': 15,
  'Plywood': 2,
  'Stone': 11,
  'Stucco': 9,
  'VinylSd': 0,
  'Wd Sdng': 1,
  'Wd Shng': 3},
 'Neighborhood': {'Blmngtn': 22,
  'Blueste': 24,
  'BrDale': 14,
  'BrkSide': 2,
  'ClearCr': 1,
  'CollgCr': 0,
  'Crawfor': 6,
  'Edwards': 3,
  'Gilbert': 10,
  'IDOTRR': 19,
  'MeadowV': 12,
  'Mitchel': 8,
  'NAmes': 7,
  'NPkVill': 20,
  'NWAmes': 15,
  'NoRidge': 18,
  'NridgHt': 16,
  'OldTown': 13,
  'SWISU': 4,
  'Sawyer': 5,
  'SawyerW': 17,
  'Somerst': 11,
  'StoneBr': 21,
  'Timber': 9,
  'Veenker': 23}}

In [39]:
# esta es la lista de variables que el codificador necesita transformar
ordinal_enc.variables

['Neighborhood', 'Exterior1st', 'Exterior2nd']

In [40]:
X_train = ordinal_enc.transform(X_train)
X_test = ordinal_enc.transform(X_test)

# exploremos el resultado
X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,0,0,0
682,1,1,1
960,2,1,2
1384,3,2,3
1100,4,1,1


**Nota**

Si el argumento 'variables' se fija en 'None' (ninguno). entonces el codificador automáticamente indentificará  **todas las variables categóricas**. Maravilloso verdad?

El codificador no codificará las variables numéricas. Entonces si algunas de tus variables categóricas son de hecho numéricas, necesitas hacer el 're-cast' o cambio a tipo 'object' antes de usar el codificador.

Si hay una variable en el set de prueba, para el cual el codificador no tiene un número para asignar ( la categoría no estaba presente en el set de entrenamiento), el codificador devolverá un error.
